### Introduction
This notebook provides an idea of how we can leverage Azure AI services to build PoCs. The goal is to provide an understanding on how a Data Scientist/ Cloud Solution Architect can solve a real-world problem into an artificial intelligence solution.

### Use case
Knowledge bases in enterprises are very common in the industry today and can have extensive number of documents in different categories. Retrieving relevant content based on a user query is a challenging task. This notebook walks through on how leverage Azure OpenAI models to create embeddings from the documents, store in Redis Cache, and leverage vector similarity search to extract text based on query.

### Step-by-step process flow

1. All files are uploaded to Storage account bucket 'nlppapers'
2. For each file, call Form Recognizer to get text contents. (We can use other OCR tools. Form recognizer gives you the flexibility to extract text from complex document templates)
3. \[optional\] Store contents of each files at .txt in 'nlppapers_text' container
4. \[optional\] For each file in 'nlppapers_text', call OpenAI.embeddings after chunking
5. Store these embeddings in Redis Cache
6. Use Vector Similarity Search feature of Redis Cache
7. Extract top k similar embeddings based on the query
8. Prompt engineer to set-up QnA based on passage (passage being top k similar results extracted from Redis Cache)

### Prerequisite - 
**For demo purposes, run redis locally**

docker run -d --name redis --rm -p 6379:6379 redislabs/redismod dislabs/redismod

## Installing relevant modules

In [1]:
%pip install redis openai azure-ai-formrecognizer azure-storage-blob

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

embedding_deployment_name = 'text-embedding-ada-002'
deployment_name = 'gpt-35-turbo-instruct'

### Extracts text using form recognizer 

In [26]:
""" This code sample shows Prebuilt Read operations with the Azure Form Recognizer client library. 
We can use other OCR tools. Form recognizer provides flexibility to extract text from complex document templates."""

import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

def get_content(document_url):
    """Returns the text content of the file at the given URL."""
    print("Analyzing", document_url)

    document_analysis_client = DocumentAnalysisClient(
        endpoint=os.environ["FORM_RECOGNIZER_ENDPOINT"],
        credential=AzureKeyCredential(os.environ["FORM_RECOGNIZER_KEY"]),
    )
    
    poller = document_analysis_client.begin_analyze_document_from_url(
            "prebuilt-read", document_url)
    result = poller.result()
    return result.content


### Process documents
Functions to
- Creates tuple of document name and authenticated URLs for documents in the given container.
- Save processed content into the given container

In [10]:
from azure.storage.blob import BlobServiceClient, BlobSasPermissions, generate_blob_sas
import datetime

container_name_documents = 'nlppapers' #raw pdf files
container_name_processed = 'nlppapers-text' #text files stored in this container

def get_authenticated_urls(container_name):
    """Returns a list of tuple of (document name, authenticated URLs) for
    documents in the given container."""

    urls = []
    # Connect to the storage account
    blob_service_client = BlobServiceClient.from_connection_string(
        os.environ["STORAGE_ACCOUNT_CONNECTION_STRING"])
    container_client = blob_service_client.get_container_client(container_name)

    # Iterate over the blobs in the container
    blob_list = container_client.list_blobs()
    for blob in blob_list:
        # Retrieve the URL of the blob
        blob_client = container_client.get_blob_client(blob.name)
        blob_url = blob_client.url

        print(f"Generating authenticated URL for: {blob.name}")

        blob_sas = generate_blob_sas(
            account_name=container_client.account_name,
            account_key=container_client.credential.account_key,
            container_name=container_name,
            blob_name=blob.name,
            permission=BlobSasPermissions(read=True),
            expiry=datetime.datetime.utcnow() + datetime.timedelta(hours=1))

        authenticated_url = f"{blob_url}?{blob_sas}"
        urls.append((blob.name, authenticated_url))
    return urls


def save_content_to_file(container_name, filename, content):
    # Connect to the storage account
    blob_service_client = BlobServiceClient.from_connection_string(
        os.environ["STORAGE_ACCOUNT_CONNECTION_STRING"])
    container_client = blob_service_client.get_container_client(container_name)

    # Create a blob client using the filename as the name for the blob
    blob_client = container_client.get_blob_client(filename)

    # Upload the content to the blob
    blob_client.upload_blob(content)

### Perform light data cleaning and creating embeddings using text-embedding-ada-002 embeddings model

In [14]:
import numpy as np
from openai import AzureOpenAI
import math
import re

# Define the chunk size
CHUNK_SIZE = 1000

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model=embedding_deployment_name): 
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# Perform light data cleaning (removing redudant whitespace and cleaning up punctuation)
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", " ")
    s = s.strip()
    
    return s

def create_embeddings(text):
    """Splits the text into chunks and returns a list of (chunk text, embeddings)."""
    # Calculate the number of chunks
    num_chunks = math.ceil(len(text) / CHUNK_SIZE)

    # Initialize an empty list to store the embeddings
    embeddings = []

    # Loop over the chunks of text
    for i in range(num_chunks):
        start = i * CHUNK_SIZE
        end = (i + 1) * CHUNK_SIZE

        chunk_text = text[start:end]
        embedding = get_embedding(normalize_text(chunk_text), model = embedding_deployment_name)
        embeddings.append((chunk_text, embedding))
    return embeddings
    
def create_embeddings_for_query(query):
    """get embeddings for the given query."""
    return get_embedding(normalize_text(query), model = embedding_deployment_name)

### Setting up connection to Redis Cache

In [18]:
from redis import Redis
from redis.commands.search.field import VectorField
from redis.commands.search.field import TextField
from redis.commands.search.field import TagField
from redis.commands.search.query import Query
from redis.commands.search.result import Result

redis_conn = Redis(host = os.getenv("REDIS_HOST"), port = os.getenv("REDIS_PORT"))
print ('Connected to redis')
# Verify connection to Azure Cache for Redis.
redis_conn.ping()

Connected to redis


### Create redis index

In [20]:
def create_hnsw_index (redis_conn, vector_field_name, initial_size, vector_dimensions=512, distance_metric='COSINE'):
    redis_conn.ft().create_index([
        VectorField(vector_field_name, "HNSW", {
            "TYPE": "FLOAT32", 
            "DIM": vector_dimensions,
            "DISTANCE_METRIC": distance_metric,
            "INITIAL_CAP": initial_size,
            }),
        TextField("document"),
        TextField("chunk"),   
        TextField("text"),   
    ])

### Storing the embeddings in redis

In [22]:
import numpy as np
import uuid

def save_to_redis(client:Redis, vector_field_name, document_name, chunk_embeddings):
    p = client.pipeline(transaction=False)
    for index, (chunk, embeddings) in enumerate(chunk_embeddings):   
        #hash key
        key = str(uuid.uuid4())

        #hash values
        embedding_binary = np.array(embeddings).astype(np.float32).tobytes()
        metadata = {
            'document': document_name,
            'chunk': str(index),
            'text': chunk,
            vector_field_name: embedding_binary,
        }
        # HSET
        # print(metadata['document'], str(index), str(embedding_binary))
        p.hset(key, mapping=metadata)
            
    p.execute()

### Get a list of documents from Azure blob storage and their authenticated urls.

In [28]:
# Get the list of all documents and their authenticated URLs. 
document_urls = get_authenticated_urls(container_name_documents)

documents = []

for (document, url) in document_urls:
    # Get text contents of each document using Form Analyzer.
    content = get_content(url)
    # Save contents to Storage account container.
    save_content_to_file(container_name_processed, document+".txt", content)
    # And create embeddings for them.

    # list of [(chunk_text, embeddings)]
    chunk_embeddings = create_embeddings(content)

    documents.append({
        'document_name': document,
        'chunk_embeddings': chunk_embeddings,
    })

Generating authenticated URL for: 1706.03762.pdf
Analyzing https://aaisea.blob.core.windows.net/nlppapers/1706.03762.pdf?se=2024-01-31T12%3A33%3A09Z&sp=r&sv=2023-11-03&sr=b&sig=0M1VelsKM/BngPww8nr1RebB0kU7vkdnkQtwx2/FfVY%3D


### Create embedding index in Redis Cache

In [29]:
ITEM_KEYWORD_EMBEDDING_FIELD='embeddings'
INITIAL_SIZE=1000
TEXT_EMBEDDING_DIMENSION=1536

redis_conn.flushall()
create_hnsw_index(redis_conn, ITEM_KEYWORD_EMBEDDING_FIELD, INITIAL_SIZE, TEXT_EMBEDDING_DIMENSION, 'COSINE')
# Verify the index created
redis_conn.ft().info()

In [31]:
for document in documents:
    print("Saving to redis: ", document['document_name'])
    save_to_redis(
        redis_conn,
        ITEM_KEYWORD_EMBEDDING_FIELD,
        document['document_name'],
        document['chunk_embeddings'])

Saving to redis:  1706.03762.pdf


In [33]:
def answer_from_chunk(chunk, query):
    prompt = """Answer the question: "{}"
    
    Only use the following passage: {}
    """.format(chunk, query)

    response = client.completions.create(
        model=deployment_name,
        prompt=prompt,
        temperature=0,
        max_tokens=300
    )
    return response

In [46]:
def answer_question(query, topK=1):
    query_embeddings = np.array(create_embeddings_for_query(query)).astype(np.float32).tobytes()

    #prepare the query
    q = Query(f'*=>[KNN {topK} @{ITEM_KEYWORD_EMBEDDING_FIELD} $vec_param AS vector_score]').sort_by('vector_score').paging(0,topK).return_fields('vector_score','document','text').dialect(2)
    params_dict = {"vec_param": query_embeddings}


    #Execute the query
    results = redis_conn.ft().search(q, query_params = params_dict)

    response = None
    # Return the first answer
    for chunk in results.docs:
        print(chunk.id, chunk.document, chunk.text)
        answer = answer_from_chunk(chunk.text, query)
        return answer.choices[0]

    print("Response not found.")

In [47]:
answer = answer_question('What is attention')
print(answer.text)

ebeff5b5-fbf8-4dcd-822d-503c8e616fa8 1706.03762.pdf ensures that the predictions for position i can depend only on the known outputs at positions less than i.
3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum
3
Scaled Dot-Product Attention
1
MatMul
1
SoftMax
Multi-Head Attention
1
Linear
Concat
Mask (opt.)
A
Scale
1
MatMul
1
1
Q K
Scaled Dot-Product
h
Attention
Linear
V
Linear
Linear
V
K
Q
Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel.
of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.
3.2.1 Scaled Dot-Product Attention
We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of queries and keys of dimension dk, and v

In [41]:
answer = answer_question('What is the multi-head attention')
print(answer.text)


Multi-head attention is a mechanism used in the Transformer model that allows for joint attention to be applied to information from different representation subspaces at different positions. It involves concatenating multiple attention heads and projecting them to obtain final values. This approach is more effective than using a single attention head, as it allows for more diverse and comprehensive information processing. In the Transformer, multi-head attention is used in three different ways, including in "encoder-decoder attention" layers where the queries come from the previous decoder layer and the memory keys and values come from the encoder. This allows for effective information transfer between the encoder and decoder layers.


In [42]:
answer = answer_question('What is attention mechanism in detail', topK=2)
print(answer.text)


Attention mechanism is a technique used in sequence modeling and transduction models to capture dependencies between input and output sequences without considering their distance. It allows for more efficient modeling of long-range dependencies and has become an integral part of various tasks. The Transformer model architecture relies entirely on attention mechanisms, eliminating the need for recurrent networks and allowing for more parallelization. This has led to significant improvements in translation quality and reduced training time. Other models, such as Extended Neural GPU, ByteNet, and ConvS2S, also aim to reduce sequential computation.


In [43]:
answer = answer_question('What is attention mechanism?')
print(answer.text)


Attention mechanism is a technique used in sequence modeling and transduction models to model dependencies between input and output sequences without considering their distance. It allows for more parallelization and has been shown to improve model performance in various tasks. The Transformer model architecture relies entirely on attention mechanisms, eschewing recurrence and achieving state-of-the-art results in translation quality with significantly less training time.


In [44]:
answer = answer_question('How to train self-attention model?')
print(answer.text)


To train a self-attention model, the first step is to gather a dataset of input-output pairs. This dataset should contain sequences of varying lengths, as the model will need to be able to handle inputs of different sizes. Next, the model is trained using a process called backpropagation, where the model's parameters are adjusted based on the error between the predicted output and the actual output. This process is repeated for multiple epochs until the model's performance on a validation set reaches a satisfactory level. Once the model is trained, it can be used to extrapolate to sequence lengths longer than the ones encountered during training. This is one of the key advantages of self-attention layers, as they are able to handle inputs of any length.


### Conclusion
This notebook is designed to showcase how a data scientist can leverage Azure OpenAI service with other Azure services like Form Recognizer, Redis Cache, Azure Blob Storage to create PoCs based on the customer requirements.
We can do engineering at various steps to customize this PoC based on the customer requirement, document corpus, and prompt engineering, chunk size etc.